In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, ttest_rel, wilcoxon

# Load and clean the data
file_path = '無題のフォーム（回答） - シート1 (1).csv'
file_path = '無題のフォーム（回答） - フォームの回答 1 (1).csv'
file_path = '無題のフォーム（回答） - シート1 (3).csv'
data = pd.read_csv(file_path)
data.columns = data.columns.str.strip().str.replace('\n', '').str.replace(' ', '')

# Extract relevant columns
data_cleaned = data[[
    '実験手順', 
    '実験番号',
    'どれくらい酔ったか？', 
    'どれくらいターゲットを取得するのが大変だったか？',
    'どれくらい視野外物体の区別をするのが難しかったか？',
    '精神的にしんどかったですか？', 
    'どれくらい体験として楽しかったか？'
]]

data_cleaned.columns = [
    'Condition',     # 音あり/音無し
    'Experiment',    # 実験番号
    'Dizziness', 
    'Target Difficulty', 
    'Object Distinction Difficulty',
    'Mental Strain', 
    'Fun Experience'
]

# Get unique experiment numbers
experiment_numbers = data_cleaned['Experiment'].unique()

# Metrics for evaluation
metrics = [
    'Dizziness', 
    'Target Difficulty', 
    'Object Distinction Difficulty', 
    'Mental Strain', 
    'Fun Experience'
]

results = []

for experiment in experiment_numbers:
    # 実験番号ごとにデータを分ける
    experiment_data = data_cleaned[data_cleaned['Experiment'] == experiment]

    # 同じ被験者のペアがちゃんと対応している前提であれば、
    # 音あり / 音無し で同じ人を1行ずつ対応づける必要があります。
    # データ構造上、被験者IDがなければ厳密なペアリングができないので、
    # ここでは簡易的に「音あり」「音無し」の順番が同じ人とみなすと仮定。
    #
    # もし "参加者ID" などがデータに含まれていれば、ID でsortしてペアリングしてください。

    for metric in metrics:
        # 音あり / 音無しそれぞれの生データ
        sound_on = experiment_data[experiment_data['Condition'] == '音あり'][metric].dropna().values
        sound_off = experiment_data[experiment_data['Condition'] == '音無し'][metric].dropna().values

        # まず正規性を判断
        if len(sound_on) < 3 or len(sound_off) < 3:
            # サンプル数が少なすぎると shapiro 検定自体が動かないのでスキップ
            continue
        
        # Shapiro-Wilk (音あり)
        stat_on, p_on = shapiro(sound_on)
        # Shapiro-Wilk (音無し)
        stat_off, p_off = shapiro(sound_off)

        # 条件: p_on と p_off 両方が0.05以上 → 正規性ありとみなす
        if p_on >= 0.05 and p_off >= 0.05:
            # 対応のある t 検定
            t_stat, t_p_value = ttest_rel(sound_on, sound_off)
            results.append({
                'Experiment': experiment,
                'Metric': metric,
                'Test': 'Paired t-test',
                'Test statistic': t_stat,
                'p-value': t_p_value,
                'Interpretation': 'Significant' if t_p_value < 0.05 else 'Not significant'
            })
        else:
            # Wilcoxon signed-rank test
            w_stat, w_p_value = wilcoxon(sound_on, sound_off)
            results.append({
                'Experiment': experiment,
                'Metric': metric,
                'Test': 'Wilcoxon signed-rank test',
                'Test statistic': w_stat,
                'p-value': w_p_value,
                'Interpretation': 'Significant' if w_p_value < 0.05 else 'Not significant'
            })

# 結果をDataFrame化して表示 (必要に応じてCSV出力も可能)
results_df = pd.DataFrame(results)
print(results_df)

# 必要ならCSVに保存
results_df.to_csv("paired_test_results.csv", index=False)


ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.